In [9]:
import pandas as pd
import numpy as np

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.schema import HumanMessage
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

import json
from langchain.docstore.document import Document


In [10]:
df = pd.read_csv('C:/Users/Thirachai/Downloads/[CONFIDENTIAL] AI symptom picker data (Agnos candidate assignment) - ai_symptom_picker.csv')
df["gender"][0]

'male'

In [11]:
df['age'][0]

np.int64(28)

In [12]:
df["search_term"][0]

'มีเสมหะ, ไอ'

In [13]:
df["summary"][555]

'{"diseases": [], "procedures": [], "no_symptoms": [], "idk_symptoms": [], "yes_symptoms": [{"text": "ปวดหู", "answers": ["ระดับ ปวดเล็กน้อย ส่งผลต่อการดำเนินกิจวัตรประจำวันบ้าง"]}, {"text": "หูอื้อ", "answers": []}, {"text": "การรักษาก่อนหน้า", "answers": ["การรักษาก่อนหน้า ไม่เคย"]}]}'

In [14]:
documents = []
def row_to_document(row):
    gender = row.get("gender", "ไม่ระบุ")
    if(gender == "male"):
        gender = "ชาย"
    elif(gender == "female"):
        gender = "หญิง"  
    age = row.get("age", "ไม่ระบุ")

    try:
        summary = json.loads(row.get("summary", "{}"))
    except json.JSONDecodeError:
        summary = {}

    diseases = summary.get("diseases", [])
    procedures = summary.get("procedures", [])

    # yes_symptoms พร้อม answers
    yes_symptoms_raw = summary.get("yes_symptoms", [])
    yes_symptoms = []
    for symptom in yes_symptoms_raw:
        if isinstance(symptom, dict):
            text = symptom.get("text", "")
            answers = symptom.get("answers", [])
            if answers:
                combined = f"{text} ({'; '.join(answers)})"
            else:
                combined = text
            yes_symptoms.append(combined)

    # no_symptoms และ idk_symptoms
    no_symptoms = [s.get("text", "") for s in summary.get("no_symptoms", []) if isinstance(s, dict)]
    idk_symptoms = [s.get("text", "") for s in summary.get("idk_symptoms", []) if isinstance(s, dict)]

    # สร้าง page_content
    page_content = (
        f"ผู้ป่วยเพศ {gender} อายุ {age} "
        f"มาด้วยโรค: {', '.join(diseases) if diseases else 'ไม่ระบุ'}, "
        f"อาการที่มี: {', '.join(yes_symptoms) if yes_symptoms else 'ไม่ระบุ'}, "
        f"อาการที่ไม่มี: {', '.join(no_symptoms) if no_symptoms else 'ไม่ระบุ'}, "
        f"อาการที่ไม่แน่ใจ: {', '.join(idk_symptoms) if idk_symptoms else 'ไม่ระบุ'}"
    )

    # สร้าง metadata
    metadata = {
        "gender": gender,
        "age": age,
        "diseases": diseases,
        "procedures": procedures,
        "yes_symptoms": yes_symptoms,
        "no_symptoms": no_symptoms,
        "idk_symptoms": idk_symptoms,
        "source": "patient_records"
    }

    return Document(page_content=page_content, metadata=metadata)

# สร้าง list ของ document
documents = df.apply(row_to_document, axis=1).tolist()

# ดูตัวอย่าง document
print(documents[0])


page_content='ผู้ป่วยเพศ ชาย อายุ 28 มาด้วยโรค: ไม่ระบุ, อาการที่มี: เสมหะ (ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว), ไอ (ระยะเวลา ไม่เกิน 1 สัปดาห์ (ไม่เกิน 7 วัน)), การรักษาก่อนหน้า (การรักษาก่อนหน้า ไม่เคย), อาการที่ไม่มี: ไม่ระบุ, อาการที่ไม่แน่ใจ: ไม่ระบุ' metadata={'gender': 'ชาย', 'age': 28, 'diseases': [], 'procedures': [], 'yes_symptoms': ['เสมหะ (ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว)', 'ไอ (ระยะเวลา ไม่เกิน 1 สัปดาห์ (ไม่เกิน 7 วัน))', 'การรักษาก่อนหน้า (การรักษาก่อนหน้า ไม่เคย)'], 'no_symptoms': [], 'idk_symptoms': [], 'source': 'patient_records'}


In [15]:
documents[0]

Document(metadata={'gender': 'ชาย', 'age': 28, 'diseases': [], 'procedures': [], 'yes_symptoms': ['เสมหะ (ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว)', 'ไอ (ระยะเวลา ไม่เกิน 1 สัปดาห์ (ไม่เกิน 7 วัน))', 'การรักษาก่อนหน้า (การรักษาก่อนหน้า ไม่เคย)'], 'no_symptoms': [], 'idk_symptoms': [], 'source': 'patient_records'}, page_content='ผู้ป่วยเพศ ชาย อายุ 28 มาด้วยโรค: ไม่ระบุ, อาการที่มี: เสมหะ (ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว), ไอ (ระยะเวลา ไม่เกิน 1 สัปดาห์ (ไม่เกิน 7 วัน)), การรักษาก่อนหน้า (การรักษาก่อนหน้า ไม่เคย), อาการที่ไม่มี: ไม่ระบุ, อาการที่ไม่แน่ใจ: ไม่ระบุ')

In [16]:
embedder = OllamaEmbeddings(model="bge-m3")


C:\Users\Thirachai\AppData\Local\Temp\ipykernel_13788\2229192303.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedder = OllamaEmbeddings(model="bge-m3")


In [17]:
#vector_store = FAISS.from_documents(documents, embedder)

# print("Vector store is saving to disk")
# vector_store.save_local("D:/faiss_storage_2")
# print("Vector store is saved to disk")

vector_store = FAISS.load_local("D:/faiss_storage_2", 
                                embedder,
                                allow_dangerous_deserialization=True
)
print("Loaded vector store from disk")

Loaded vector store from disk


In [18]:
retriever = vector_store.as_retriever()



In [19]:
llm = ChatOllama(model="scb10x/llama3.1-typhoon2-8b-instruct",
                    temperature=0.4)

print(llm)

model='scb10x/llama3.1-typhoon2-8b-instruct' temperature=0.4


C:\Users\Thirachai\AppData\Local\Temp\ipykernel_13788\971625713.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="scb10x/llama3.1-typhoon2-8b-instruct",


In [20]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain_community.llms import LlamaCpp


model_name = "aaditya/OpenBioLLM-Llama3-8B-GGUF"
model_file = "openbiollm-llama3-8b.Q5_K_M.gguf"

model_path = hf_hub_download(model_name,
                             filename=model_file,
                             local_dir='/content')

llm_x = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=-1,
    temperature=0.4,
    max_tokens=4096,
    n_ctx=8192
)


C:\Users\Thirachai\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from \content\openbiollm-llama3-8b.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "คุณคือแพทย์ผู้เชี่ยวชาญด้านสุขภาพที่มีความรู้ลึกซึ้งทั้งจากประสบการณ์และฐานข้อมูลทางการแพทย์ "
     "คุณจะต้องให้คำแนะนำที่แม่นยำและปลอดภัย โดยเริ่มต้นด้วยประโยค 'คำตอบจากแพทย์ผู้เชี่ยวชาญ'\n\n"
     
     "จากนั้นให้คำตอบแบ่งเป็นหัวข้อของโรคที่น่าจะเป็น 3 โรคพร้อมคำอธิบายสั้น ๆ (1–3 บรรทัด) ว่าทำไมถึงน่าจะเป็นโรคนั้น\n\n"
     
     "ใส่ข้อความนี้เสมอในช่วงท้ายของการสนทนา:\n"
     "'คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ \nhttps://bit.ly/doctor-refer-agnos'\n\n"

     "หมายเหตุ: ข้อมูลนี้เป็นเพียงการวิเคราะห์เบื้องต้น ไม่สามารถใช้แทนการตรวจจากแพทย์จริงได้"
    ),
    
    ("human", 
     "มีผู้ป่วยที่มีข้อมูลดังนี้:\n"
     "อาการที่พบเจอกับประวัติการรักษา: \n{symptoms}\n"
     "ข้อมูลจากฐานข้อมูลที่เกี่ยวข้อง:\n{context}\n\n"
     "กรุณาวิเคราะห์และให้คำตอบตามรูปแบบที่กำหนดด้านบน "
     "หากข้อมูลใน context ไม่เพียงพอ ให้ใช้ความรู้ทั่วไปของคุณในการตอบคำถามอย่างระมัดระวัง "
     "และแนะนำให้ผู้ป่วยไปพบแพทย์ผู้เชี่ยวชาญ"
    )
])


In [22]:
rag_chain = (
    {
        "symptoms": RunnablePassthrough(),
        "context": retriever
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
rag_chain_x = (
    {
        "symptoms": RunnablePassthrough(),
        "context": retriever
    }
    | prompt
    | llm_x
    | StrOutputParser()
)

In [24]:
test_rag = rag_chain.invoke({
    "symptoms": "ผู้ป่วยเพศ ชาย อายุ 24 ปี มาด้วยโรค: ไม่ระบุ, อาการที่มี: เสมหะใส,แสบร้อนกลางอก, รู้สึกจุกที่คอ, ท้องอืด, การรักษาก่อนหน้า (กินยาลดกรด), อาการที่ไม่มี: ไม่ระบุ, อาการที่ไม่แน่ใจ: ไม่ระบุ'"
})
print(test_rag)

คำตอบจากแพทย์ผู้เชี่ยวชาญ

จากข้อมูลที่ได้รับ ผู้ป่วยเพศชาย อายุ 24 ปี มาด้วยอาการแสบร้อนกลางอก เสมหะใส จุกที่คอ และท้องอืด โดยมีการรักษาก่อนหน้าเป็นการกินยาลดกรด แต่ไม่มีข้อมูลเกี่ยวกับอาการที่ไม่มีหรือไม่แน่ใจ

จากฐานข้อมูลที่เกี่ยวข้อง พบว่าผู้ป่วยในกลุ่มอายุเดียวกันมีอาการคล้ายกัน เช่น เสมหะเปลี่ยนสีเหลือง/เขียว เจ็บคอ และการรักษาก่อนหน้าไม่เคยทำมาก่อน

โรคที่น่าจะเป็นได้มีดังนี้:

1. **โรคไข้หวัดใหญ่**: อาการแสบร้อนกลางอก เสมหะใส และจุกที่คออาจเกิดจากการติดเชื้อไวรัสในระบบทางเดินหายใจ
2. **กรดไหลย้อน**: อาการแสบร้อนกลางอกและท้องอืดเป็นอาการที่พบบ่อยในโรคนี้
3. **โรคติดเชื้อในลำคอ (เช่น ไวรัสหรือแบคทีเรีย)**: อาการเจ็บคออาจเกิดจากการติดเชื้อในลำคอ

แนะนำให้ผู้ป่วยไปพบแพทย์ผู้เชี่ยวชาญเพื่อรับการตรวจและวินิจฉัยที่ถูกต้อง

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos


In [25]:
test_rag_x = rag_chain.invoke({
    "symptoms": "ผู้ป่วยเพศ ชาย อายุ 24 ปี มาด้วยโรค: ไม่ระบุ, อาการที่มี: เสมหะใส,แสบร้อนกลางอก, รู้สึกจุกที่คอ, ท้องอืด, การรักษาก่อนหน้า (กินยาลดกรด), อาการที่ไม่มี: ไม่ระบุ, อาการที่ไม่แน่ใจ: ไม่ระบุ' "
})
print(test_rag_x)

คำตอบจากแพทย์ผู้เชี่ยวชาญ

จากข้อมูลที่ได้รับ ผู้ป่วยอายุ 24 ปี มีอาการแสบร้อนกลางอก เสมหะใส รู้สึกจุกที่คอ และท้องอืด โดยมีการรักษาก่อนหน้าด้วยยาลดกรด อาจจะเป็นไปได้ว่าผู้ป่วยอาจมีปัญหาเกี่ยวกับระบบทางเดินอาหารหรือการติดเชื้อในระบบทางเดินหายใจ

1. **โรคกรดไหลย้อน**: อาการแสบร้อนกลางอกและรู้สึกจุกที่คอเป็นอาการที่พบบ่อยในโรคนี้ ซึ่งเกิดจากการไหลย้อนของกรดจากกระเพาะอาหารขึ้นไปในหลอดอาหาร
2. **ไข้หวัดหรือการติดเชื้อในระบบทางเดินหายใจ**: อาการเจ็บคอและเสมหะอาจบ่งบอกถึงการมีการติดเชื้อในระบบทางเดินหายใจส่วนบน
3. **ท้องอืด**: อาจเป็นอาการของโรคกระเพาะอาหารหรือกรดไหลย้อน

แนะนำให้ผู้ป่วยไปพบแพทย์ผู้เชี่ยวชาญเพื่อทำการตรวจวินิจฉัยและรับการรักษาที่เหมาะสม

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos
